![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Multimodal - Video
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)

# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']


# 1 | Einführung

---



Video spielt eine zentrale Rolle in vielen Anwendungen, sei es in der Inhaltsanalyse, der automatischen Objekterkennung oder der Generierung von Inhalten. Durch moderne KI-Technologien können Videoinhalte effizient verarbeitet und ausgewertet werden. In diesem Abschnitt werden vier wesentliche Anwendungsbereiche der Videoverarbeitung vorgestellt:

- **Video-zu-Text (Video-to-Text, VTT)**: Videoinhalte werden in Text umgewandelt, um visuelle Szenen, Handlungen oder Präsentationen schriftlich festzuhalten. Dies erleichtert die Dokumentation und Weiterverarbeitung von visuellen Inhalten.
- **Videoanalyse**: Durch die Analyse von Bildinhalten, Bewegungen, Objekten und Szenen können relevante Informationen aus Videos extrahiert werden. Diese Methode wird häufig in der Sicherheitsüberwachung, im Marketing oder in der Forschung eingesetzt.
- **Video-Summary**: Wichtige Informationen aus langen Videodateien werden extrahiert und in einer kompakten Form dargestellt. Dies ermöglicht eine schnelle Erfassung der Kernaussage, ohne das gesamte Material ansehen zu müssen.
- **Bild-zu-Video (Image-to-Video, ITV)**: Statische Bilder werden in bewegte Videosequenzen umgewandelt. Diese Technik wird für die Animation von Illustrationen, die Erstellung dynamischer Erklärvideos oder die visuelle Inszenierung von Konzepten genutzt.

Diese Techniken erleichtern die Verarbeitung und Nutzung großer Mengen visueller Inhalte und eröffnen neue Möglichkeiten für die Automatisierung und Analyse von Video-Daten.




# 2 | Video-to-Text (VTT)

---



Die Umwandlung von Videoinhalten in Text, bekannt als Video-Transkription, ermöglicht die automatische Verschriftlichung von Videoszenen, Vorträgen oder Handlungen. Dies erleichtert die Dokumentation, Archivierung und Weiterverarbeitung von Videoinhalten. Moderne Bilderkennung und Spracherkennungstechnologien nutzen KI-Modelle, um visuelle und auditive Elemente präzise zu erfassen und in lesbaren Text umzuwandeln.

Videotranskriptionen sind in vielen Bereichen von Bedeutung – von der Erstellung von Untertiteln und der Zugänglichkeit für Menschen mit Behinderungen über die Erstellung durchsuchbarer Videoinhalte bis hin zur Analyse von Marktforschungsdaten. Durch den Einsatz automatisierter Systeme kann dieser Prozess erheblich beschleunigt werden, wodurch große Mengen an Videomaterial effizient verarbeitet werden können.

<p><font color='black' size="5">
Version auf Basis Transkript
</font></p>

In [ ]:
# Import
import openai
from google.colab import files
from IPython.display import display, Markdown
import os

# Nutzer wählt eine Datei aus und lädt sie hoch
uploaded = files.upload()

# Den ersten Dateinamen extrahieren
video_file_path = list(uploaded.keys())[0]

# Temporären Pfad für die Audio-Extraktion erstellen
audio_path = "extracted_audio.mp3"

# FFmpeg verwenden, um Audio aus dem Video zu extrahieren
!ffmpeg -i "{video_file_path}" -q:a 0 -map a "{audio_path}" -y

# Audiodatei öffnen und Whisper nutzen
with open(audio_path, "rb") as audio_file:
    response = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )

In [ ]:
# Markdown-Formatierte Ausgabe
markdown_output = f"""
## 🎥 Transkript des Videos

**📂 Datei:** `{video_file_path}`
**🤖 Modell:** `Whisper-1` (für Audiotranskription)

---

{response.text}
"""

# Markdown in Google Colab anzeigen
display(Markdown(markdown_output))

# Temporäre Audiodatei löschen
# os.remove(audio_path)

<p><font color='black' size="5">
Version auf Basis Bilder
</font></p>

In Anlehnung an: [Quelle](https://cookbook.openai.com/examples/gpt_with_vision_for_video_understanding).

In [ ]:
from IPython.display import display, Image, Markdown
from google.colab import files
import cv2
import base64
import time
from openai import OpenAI
import os
import requests

client = OpenAI()

In [ ]:
# Datei-Upload ermöglichen
uploaded = files.upload()  # Öffnet einen Dialog zum Datei-Upload

# Name der hochgeladenen Datei ermitteln
filename = list(uploaded.keys())[0]
print(f"Datei '{filename}' wurde hochgeladen.")

# Video mit OpenCV öffnen
video = cv2.VideoCapture(filename)

# Frames in base64 umwandeln
base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

# Video-Ressourcen freigeben
video.release()
print(len(base64Frames), "Frames wurden eingelesen.")

In [ ]:
# Prompt und Chat-Aufruf
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video that I want to upload. Generate a compelling description that I can upload along with the video. Please answer in German.",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
        ],
    },
]
params = {
    "model": "gpt-4o-mini",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 200,
}

result = client.chat.completions.create(**params)

In [ ]:
# Markdown-Formatierte Ausgabe
markdown_output = f"""
## 🎥 Analyse des Videos auf Basis Frames

**📂 Datei:** `{filename}`
**🤖 Modell:** gpt-4o-mini

---
"""

display(Markdown(markdown_output))
display(Markdown(result.choices[0].message.content))

# 3 | Image-to-Video (I2V)

---

Image-zu-Video (I2V) mit generativer KI ermöglicht die Erstellung realistischer Videos auf Basis einzelner Bilder oder Bildsequenzen. Modelle wie Sora (OpenAI), Gen-2 (Runway) und Lumiere (Google) verwandeln statische Bilder in dynamische, visuell konsistente Videoclips – häufig ergänzt durch passende Audiospuren oder die Möglichkeit, eigenen Ton hinzuzufügen.

Diese KI-Systeme nutzen multimodale Architekturen und Deep Learning, um Bewegungen, Kamerafahrten und visuelle Effekte glaubwürdig zu simulieren. Anwendungsfelder sind u. a. Marketing, Bildung, digitale Kunst und Simulation.

Führende I2V-Modelle:

+ Sora (OpenAI) – Erzeugt aus Bildern und Bildfolgen realistische Szenen mit hoher Kohärenz.

+ Gen-2 (Runway) – Unterstützt Bild-, Text- und Hybridprompts für vielseitige Anwendungen.

+ Pika Labs – Ideal für einfache Animationen aus Einzelbildern.

+ Stable Video Diffusion – Quelloffen, mit wachsender Community.

+ Lumiere (Google) – Experimentell, mit Fokus auf Bewegungsdynamik und Stilübertragung.

+ Hedra – Ermöglicht die Erstellung von Videos aus statischen Bildern mit synchronisiertem Audio, ideal für die Generierung sprechender oder singender Avatare.

+ HeyGen – Spezialisiert auf die schnelle Erstellung personalisierter, sprechender Avatare aus Fotos oder Videos. Bietet präzise Lippensynchronisation, Voice-Cloning, Text-zu-Video und Videoübersetzung in über 40 Sprachen. Besonders geeignet für Marketing, E-Learning und Social Media; ermöglicht Multi-Avatar-Videos und individuelle Anpassungen bis hin zu 4K-Qualität.


Viele Tools bieten zusätzlich Funktionen zur Audiointegration, etwa durch automatische Tonunterlegung oder individuelle Sprachkommentare, um das Ergebnis noch lebendiger zu gestalten.

Hier ist eine Übersicht, welche der genannten Modelle mit/ohne **API-Zugang** nutzbar sind:

| Modell                   | API-Zugang möglich?               | Hinweise |
| :--                      | :--                              | :-- |
| **Sora (OpenAI)**        | Eingeschränkt, nur nach Bewerbung | Sora ist nicht öffentlich per API verfügbar. Zugang gibt es nur nach individueller Bewerbung und Freischaltung durch OpenAI. |
| **Gen-2 (Runway)**       | Ja                               | Runway bietet eine API für Gen-3 und Gen-4 Modelle, die für Entwickler offen zugänglich ist. |
| **Pika Labs**            | Ja                               | Die Pika API ist verfügbar, allerdings mit bestimmten Einschränkungen und nach Registrierung. |
| **Stable Video Diffusion** | Ja                             | Über die Developer Platform von Stability AI ist eine API für Stable Video Diffusion verfügbar. |
| **Lumiere (Google)**     | Nein, aktuell nicht              | Lumiere befindet sich noch in der Entwicklung und ist nicht öffentlich oder per API nutzbar. |
| **Hedra**                | Nein, aktuell nicht              | Hedra ist als Web-App nutzbar, aber es gibt derzeit keine öffentlich dokumentierte API. |
| **HeyGen**               | Ja                               | HeyGen bietet einen dokumentierten API-Zugang für Video- und Avatar-Erstellung. Authentifizierung erfolgt per API-Key oder OAuth2. Die API ist für Entwickler offen zugänglich, Details in der offiziellen Doku[1][3][8]. |


**Zusammengefasst:**  
Per API lassen sich aktuell **Gen-2 (Runway)**, **Pika Labs**, **Stable Video Diffusion** und **HeyGen** direkt nutzen.  
**Sora** ist nur nach individueller Bewerbung und Freischaltung per API zugänglich.  
**Lumiere** und **Hedra** bieten derzeit keinen öffentlichen API-Zugang.


Stand: 05.2025

[Hedra](https://www.hedra.com/app/video)

[HeyGen](https://app.heygen.com/get-started)

![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/under_construction_dall_e_klein.png)

[Diffuser](https://huggingface.co/docs/diffusers/v0.33.1/en/index)


[Videogeneration](https://huggingface.co/docs/diffusers/using-diffusers/text-img2vid?popular-models=CogVideoX)

In [ ]:
!uv pip install --system --prerelease allow -q -U diffusers transformers accelerate

In [ ]:
!uv pip install --system --prerelease allow -q --upgrade torchvision
!pip install torchvision --no-cache-dir --force-reinstall "torchvision-cuda==11.8"

**Restart Kernel**  `Ctrl+M.`

In [ ]:
!uv pip install --system --prerelease allow -q --upgrade torchvision
!uv pip install --system --prerelease allow -q --upgrade torch

Stable Diffusion Video also accepts micro-conditioning, in addition to the conditioning image, which allows more control over the generated video:

+ `fps`: the frames per second of the generated video.
+ `motion_bucket_id`: the motion bucket id to use for the generated video. This can be used to control the motion of the generated video. Increasing the motion bucket id increases the motion of the generated video.
+ `noise_aug_strength`: the amount of noise added to the conditioning image. The higher the values the less the video resembles the conditioning image. Increasing this value also increases the motion of the generated video.

For example, to generate a video with more motion, use the motion_bucket_id and noise_aug_strength micro-conditioning parameters:

In [ ]:
# Import
import torch

# Importiere die benötigten Module aus diffusers
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

In [ ]:
# Lade die vortrainierte Stable Video Diffusion Pipeline
pipe = StableVideoDiffusionPipeline.from_pretrained(
  "stabilityai/stable-video-diffusion-img2vid-xt",  # Modellname von Hugging Face
  torch_dtype=torch.float16,                        # Reduzierter Speicherbedarf durch Verwendung von float16
  variant="fp16"                                    # Modellvariante für float16-Unterstützung
)

In [ ]:
# Optional: Verschiebe Modellkomponenten automatisch zwischen CPU und GPU bei Bedarf (auskommentiert)
# pipe.enable_model_cpu_offload() - > Führt zu einem Fehler

In [ ]:
# Lade ein Bild von einer URL als Ausgangspunkt für die Videoerzeugung
image = load_image("https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/a_retro-futuristic_robot_dall_e.jpg")
image = image.resize((1024, 576))  # Passe die Bildgröße an das Modellinput-Format an

In [ ]:
# Erzeuge ein Video basierend auf dem Eingabebild
frames = pipe(
    image,                          # Eingabebild
    decode_chunk_size=8,            # Anzahl der Frames, die gleichzeitig decodiert werden (Speicheroptimierung)
    generator=generator,            # Zufallsgenerator für Reproduzierbarkeit
    motion_bucket_id=180,           # Steuert die Bewegung im Video (höher = mehr Bewegung)
    noise_aug_strength=0.1          # Rauschstärke während der Augmentation (steigert Variation)
).frames[0]                         # Extrahiere die erste Liste von Frames (ein Video)

In [ ]:
# Exportiere die Frames als Video-Datei mit 7 Bildern pro Sekunde
export_to_video(frames, "output_itv.mp4", fps=7)

# 4 | Text-to-Video (T2V)

---

Text-zu-Video (T2V) mit generativer KI ermöglicht die Erstellung realistischer Videos allein auf Basis von Texteingaben. Modelle wie Sora (OpenAI), Gen-2 (Runway) und Lumiere (Google) wandeln sprachliche Beschreibungen in dynamische, visuell konsistente Videoclips um – oft ergänzt durch passende Audiospuren oder die Möglichkeit, eigenen Ton hinzuzufügen.

Diese KI-Systeme nutzen multimodale Architekturen und tiefes Lernen, um Bewegungen, Perspektiven und visuelle Effekte realitätsnah zu generieren. Sie finden Anwendung in Bereichen wie Marketing, Bildung, Storytelling und Simulation.

Führende T2V-Modelle:


+ Sora (OpenAI) – Erzeugt aus Bildern und Bildfolgen realistische Szenen mit hoher Kohärenz.

+ Gen-2 (Runway) – Unterstützt Bild-, Text- und Hybridprompts für vielseitige Anwendungen.

+ Pika Labs – Ideal für einfache Animationen aus Einzelbildern.

+ Stable Video Diffusion – Quelloffen, mit wachsender Community.

+ Lumiere (Google) – Experimentell, mit Fokus auf Bewegungsdynamik und Stilübertragung.

+ Hedra – Ermöglicht die Erstellung von Videos aus statischen Bildern mit synchronisiertem Audio, ideal für die Generierung sprechender oder singender Avatare.

+ HeyGen – Spezialisiert auf die schnelle Erstellung personalisierter, sprechender Avatare aus Fotos oder Videos. Bietet präzise Lippensynchronisation, Voice-Cloning, Text-zu-Video und Videoübersetzung in über 40 Sprachen. Besonders geeignet für Marketing, E-Learning und Social Media; ermöglicht Multi-Avatar-Videos und individuelle Anpassungen bis hin zu 4K-Qualität.

Viele Tools bieten zusätzlich Funktionen zur Audiointegration, etwa durch automatische Tonunterlegung oder individuelle Sprachkommentare, um das Ergebnis noch lebendiger zu gestalten.



Hier ist eine Übersicht, welche der genannten Modelle mit/ohne **API-Zugang** nutzbar sind:

| Modell                   | API-Zugang möglich?               | Hinweise |
| :--                      | :--                              | :-- |
| **Sora (OpenAI)**        | Eingeschränkt, nur nach Bewerbung | Sora ist nicht öffentlich per API verfügbar. Zugang gibt es nur nach individueller Bewerbung und Freischaltung durch OpenAI. |
| **Gen-2 (Runway)**       | Ja                               | Runway bietet eine API für Gen-3 und Gen-4 Modelle, die für Entwickler offen zugänglich ist. |
| **Pika Labs**            | Ja                               | Die Pika API ist verfügbar, allerdings mit bestimmten Einschränkungen und nach Registrierung. |
| **Stable Video Diffusion** | Ja                             | Über die Developer Platform von Stability AI ist eine API für Stable Video Diffusion verfügbar. |
| **Lumiere (Google)**     | Nein, aktuell nicht              | Lumiere befindet sich noch in der Entwicklung und ist nicht öffentlich oder per API nutzbar. |
| **Hedra**                | Nein, aktuell nicht              | Hedra ist als Web-App nutzbar, aber es gibt derzeit keine öffentlich dokumentierte API. |
| **HeyGen**               | Ja                               | HeyGen bietet einen dokumentierten API-Zugang für Video- und Avatar-Erstellung. Authentifizierung erfolgt per API-Key oder OAuth2. Die API ist für Entwickler offen zugänglich, Details in der offiziellen Doku[1][3][8]. |


**Zusammengefasst:**  
Per API lassen sich aktuell **Gen-2 (Runway)**, **Pika Labs**, **Stable Video Diffusion** und **HeyGen** direkt nutzen.  
**Sora** ist nur nach individueller Bewerbung und Freischaltung per API zugänglich.  
**Lumiere** und **Hedra** bieten derzeit keinen öffentlichen API-Zugang.


Stand: 05.2025

[Hedra](https://www.hedra.com/app/video)

[HeyGen](https://app.heygen.com/get-started)

![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/under_construction_dall_e_klein.png)

In [ ]:
# Import
import torch
from diffusers import CogVideoXImageToVideoPipeline
from diffusers.utils import export_to_video, load_image

In [ ]:
# Bild laden
image = load_image(image="/content/a_retro-futuristic_robot_dall_e.jpg")

In [ ]:
# Prompt für die Animation
prompt = """
A cheerful retro-futuristic robot arrives at a chrome-plated welcome center on a distant planet, its polished body gleaming in the warm glow of a golden sunset. Neon lights flicker above the entrance, reading "Robots Welcome" as a crowd of similarly styled robots moves inside. The robot turns its head, smiles, and waves at the camera . Inside, metallic walls reflect colorful lights, while robots converse, exchange parts, and charge up. The scene captures a vibrant utopia where robots live in harmony, animated in a smooth, cinematic style with a nostalgic 1950s sci-fi aesthetic.
"""

In [ ]:
# Importiere das CogVideoX-Pipeline-Modul (dieser Teil muss vorher erfolgt sein)
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    "THUDM/CogVideoX-5b-I2V",          # Name des vortrainierten Modells
    torch_dtype=torch.bfloat16        # Reduziere den Speicherverbrauch durch Verwendung des bfloat16-Datentyps
)

In [ ]:
# Reduziere den Speicherbedarf während der Verarbeitung
pipe.vae.enable_tiling()             # Aktiviere Tiling (Bildaufteilung in kleinere Kacheln)
pipe.vae.enable_slicing()            # Aktiviere Slicing (Schichtenweise Verarbeitung)

In [ ]:
# Verschiebe das Modell auf die CUDA-Grafikkarte für schnellere Berechnungen
pipe.to("cuda")

In [ ]:
# Erzeuge ein Video aus einem Bild und einem Prompt
video = pipe(
    prompt=prompt,                   # Textbeschreibung, die die Bild-zu-Video-Generierung steuert
    image=image,                     # Eingabebild, das als Ausgangspunkt dient
    num_videos_per_prompt=1,         # Anzahl der Videos, die für den Prompt generiert werden sollen
    num_inference_steps=50,          # Anzahl der Inferenzschritte (mehr Schritte = bessere Qualität, aber langsamer)
    num_frames=49,                   # Anzahl der Bilder (Frames) im resultierenden Video
    guidance_scale=6,                # Stärke der Prompt-Steuerung (höher = näher am Textprompt)
    generator=torch.Generator(device="cuda").manual_seed(42),  # Zufallszahlengenerator für reproduzierbare Ergebnisse
).frames[0]                          # Extrahiere das erste generierte Video (als Liste von Frames)

In [ ]:
# Export des Videos
export_to_video(video, "output_ttv.mp4", fps=8)

# 5 | Objekterkennung

---

Die Video-Objekterkennung ist ein wichtiger Bereich der Computer Vision, der sich mit der Identifikation und Klassifikation von Objekten in Videomaterial befasst. Im Gegensatz zur statischen Bilderkennung müssen dabei nicht nur einzelne Frames analysiert, sondern auch zeitliche Zusammenhänge berücksichtigt werden.  

Ziel dieses Kapitels ist es, ein grundlegendes Verständnis für die Herausforderungen und Lösungsansätze der Video-Objekterkennung zu vermitteln und praxisnahe Anwendungen zu demonstrieren. Zum Einsatz kommt **YOLO** (You Only Look Once), ein Echtzeit-Objekterkennungsmodell, das durch seine hohe Geschwindigkeit und Genauigkeit besonders für Anwendungen wie autonome Fahrzeuge, Überwachungssysteme und interaktive KI-Systeme geeignet ist.

[Ultralytics | Yolo](https://www.ultralytics.com/de)

In [ ]:
!uv pip install --system --prerelease allow -q ultralytics opencv-python

In [ ]:
# Import
from ultralytics import YOLO
from google.colab import files
import cv2
from IPython.display import HTML
from base64 import b64encode

# YOLO-Modell laden
model = YOLO("yolo11s.pt")

# Video-Datei hochladen
uploaded = files.upload()

# Dateinamen abrufen (das erste hochgeladene Video)
video_path = list(uploaded.keys())[0]

# YOLO-Vorhersage auf dem Video durchführen
# Hier werden Objekte erkannt und das Video mit Bounding Boxes gespeichert
results = model.predict(source=video_path, save=True, conf=0.25, save_txt=True, format='avi')

# Das verarbeitete Video anzeigen
video_path = video_path.replace("mp4", "avi")
processed_video_path = f"{results[0].save_dir}/{video_path}"

# Video herunterladen
print(f"Video mit Objekterkennung: {processed_video_path}")
files.download(processed_video_path)

# 6 | Video-/Bild-/Sprachanalyse

---



Durch die Analyse von Bildinhalten, Objekten, Bewegungen und Szenen können relevante Informationen aus Videos extrahiert werden. Die Fokus in diesem Modul ist die Videoanalyse mit Computer Vision, auch **Visual Analysis** genannt, sie ist eine Methode zur automatischen Erkennung und Klassifizierung von Objekten, Personen und Handlungen in Videosequenzen. Mithilfe maschinellen Sehens (Computer Vision) werden Videos auf verschiedene Elemente und Muster untersucht. Diese Technik wird häufig in der Sicherheitsüberwachung, in der Produktionslinie oder bei der Analyse von Social-Media-Inhalten eingesetzt.

Moderne KI-Modelle analysieren nicht nur einzelne Frames, sondern berücksichtigen auch zeitliche Zusammenhänge, um Bewegungen und Handlungen präziser zu erfassen. Unternehmen nutzen diese Technologie, um Kundenverhalten zu verstehen, Sicherheitsrisiken zu erkennen oder Prozesse zu optimieren.

In [ ]:
import openai
from google.colab import files
from IPython.display import display, Markdown, HTML
import cv2
import os
import numpy as np
import base64
import tempfile

In [ ]:
# Video hochladen und Frame-Extraction
uploaded = files.upload()
video_file_path = list(uploaded.keys())[0]

In [ ]:
# Frame-Extraction
def extract_frames(video_path, num_frames=5):
    """Extrahiert Frames aus einem Video"""
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames-1, num_frames, dtype=int)

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames

# Hilfsfunktion für Base64-Kodierung
def get_base64_encoded(image_path):
    """Konvertiert ein Bild in Base64-String"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Frames extrahieren und speichern
frames = extract_frames(video_file_path)
frame_paths = [f"frame_{i}.jpg" for i in range(len(frames))]
for i, frame in enumerate(frames):
    cv2.imwrite(frame_paths[i], frame)

# Audio extrahieren und transkribieren
audio_path = "extracted_audio.mp3"
!ffmpeg -i "{video_file_path}" -q:a 0 -map a "{audio_path}" -y

with open(audio_path, "rb") as audio_file:
    transcribed_text = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    ).text

# Frame-Analyse mit GPT-4o
frame_descriptions = []
for frame_path in frame_paths:
    base64_img = get_base64_encoded(frame_path)
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Du bist ein Experte für Videoanalyse. Beschreibe kurz und präzise, was auf diesem Frame zu sehen ist."},
            {"role": "user", "content": [
                {"type": "text", "text": "Beschreibe diesen Video-Frame mit einem kurzen Satz:"},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_img}"}}
            ]}
        ],
        max_tokens=150
    )
    frame_descriptions.append(response.choices[0].message.content)

In [ ]:
# Gesamtanalyse
analysis_prompt = f"""
Analysiere den folgenden Videoinhalt basierend auf dem Transkript und den Frame-Beschreibungen.
Gib eine detaillierte Zusammenfassung dessen, was im Video passiert.

Hauptpunkt: Inhaltsanalyse:
    Themen und Argumentationsstruktur: Was wird gesagt? Wie ist das Gesagte aufgebaut?
    Sprachstil: Alltagssprache vs. Fachsprache, formell vs. informell
    Rhetorische Mittel: Metaphern, Wiederholungen, Fragen, Ironie, Pausen
    Satzstruktur & Wortwahl: Einfach oder komplex? Viele Füllwörter?
    Erzählperspektive: Ich-Form, Du-Ansprache, distanziert?
    Tonfall und Haltung: Neutral, ironisch, kritisch, emotional?

Hauptpunkt: Parasprachliche Merkmale (Stimme, Ton, Sprechweise):
    Sprechtempo: Schnell, langsam, variierend
    Intonation: Monoton oder lebendig? Betonung gezielt eingesetzt?
    Lautstärke: Konstant, wechselnd, passend zur Situation?
    Stimmqualität: Klar, nasal, heiser, angespannt?
    Pausen: Natürlich oder unnatürlich? Strategisch gesetzt?
    Versprecher oder Selbstkorrekturen: Häufig? Sympathisch oder irritierend?

Hauptpunkt: Visuelle Elemente:
    Bildkomposition: Einstellungsgrößen, Perspektiven, Bildausschnitte
    Kameraführung: Statisch, dynamisch, Schwenks, Zooms
    Schnittrhythmus: Schnell, langsam, variierend, passend zum Inhalt?
    Lichtgestaltung: Natürliches vs. künstliches Licht, Farbstimmung
    Setting/Location: Bedeutung und Wirkung des Hintergrunds
    Kleidung/Styling: Formell, informell, thematisch passend?

Hauptpunkt: Nonverbale Kommunikation:
    Körperhaltung: Offen, geschlossen, angespannt, entspannt?
    Gestik: Häufigkeit, Natürlichkeit, unterstützend zum Gesagten?
    Mimik: Ausdrucksstark, zurückhaltend, authentisch?
    Blickkontakt: Direkt in die Kamera oder ablenkend?
    Bewegung im Raum: Statisch oder dynamisch?

Hauptpunkt: Zusammenspiel von Bild und Ton:
    Synchronität: Passen Bild und Ton zusammen?
    Ergänzung: Unterstützen visuelle Elemente das Gesagte?
    Musik/Soundeffekte: Einsatz und Wirkung auf die Gesamtwahrnehmung
    Grafiken/Einblendungen: Sinnvoll integriert und informativ?

Hauptpunkt: Wirkung auf die Zuschauer:innen
    Verständlichkeit: Wird der Inhalt klar transportiert?
    Authentizität: Wirkt die Person/Szene glaubwürdig und echt?
    Emotionale Ansprache: Berührt der Inhalt? Motiviert? Regt zum Denken an?
    Zielgruppenorientierung: Passt Sprache, Ton und Bild zum Publikum?

Hauptpunkt: Stimmungsanalyse:
    Stimmung: Positiv 😊, Neutral 😐, Negativ 😞
    Begründe Deine Einschätzung unter Berücksichtigung visueller und auditiver Elemente

Hauptpunkt: Technische Aspekte
    Audioqualität: Rauschfrei, Hall, Hintergrundgeräusche?
    Mikrofonierung: Richtiger Abstand, Pop-Geräusche, Aussteuerung?
    Bildqualität: Schärfe, Auflösung, Kontrast, Farbwiedergabe
    Schnitt & Nachbearbeitung: Erkennbar geschnitten? Übergänge sauber?
    Spezialeffekte/CGI: Sinnvoll eingesetzt und qualitativ hochwertig?

Hauptpunkt: Weitere Aspekte:
    Gibt es ethische Aspekte die angesprochen werden müssten?
    Gibt es versteckte Botschaften in Bild oder Ton?
    Wie ist die Konsistenz zwischen verbaler und visueller Kommunikation?
    Welche Punkte könnten noch interessant sein, an die ich nicht gedacht habe?

Vermeide unscharfe Formulierungen  wie wahrscheinlich, möglicherweise, scheint, könnte oder sollte.
Nimm konkret Stellung.

Stelle bei der Ausgabe der Ergebnisse nur bei den Hauptpunkten zur besseren Unterscheidung ein "🔹" voran.
Zeige die Unterpunkte als dot-Aufzählung an.

### Video-Transkript:
{transcribed_text}

### Frame-Beschreibungen:
{' | '.join([f"Frame {i+1}: {desc}" for i, desc in enumerate(frame_descriptions)])}
"""

In [ ]:
analysis_result = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Du bist ein Experte für Videoanalyse."},
        {"role": "user", "content": analysis_prompt}
    ],
    temperature=0.2
).choices[0].message.content

In [ ]:
# HTML für Ergebnisse-Anzeige definieren

frames_html = "".join([
    f"""
    <div style="display: inline-block; margin: 10px; text-align: center;">
        <img src="data:image/jpeg;base64,{get_base64_encoded(frame_path)}" style="width: 200px; border: 1px solid #ddd;">
        <p style="width: 200px; font-size: 12px;">{frame_descriptions[i]}</p>
    </div>
    """ for i, frame_path in enumerate(frame_paths)
    ])

# Markdown-Anzeige
display(Markdown(f"""
## 🎥 Videoanalyse
---

**📂 Datei:** `{video_file_path}`
**🤖 Modelle:** `Whisper-1` (Audio), `GPT-4o` (Frames), `GPT-4o-mini` (Gesamtanalyse)

<br>

### 📝 Transkript
---
{transcribed_text}

<br>

### 🖼️ Extrahierte Frames und Beschreibungen
---
"""))

display(HTML(frames_html))
display(Markdown(f"""
### 🧠 Videoanalyse-Ergebnis
{analysis_result}
"""))

In [ ]:
# Aufräumen - Dateien löschen
for path in frame_paths + [audio_path]:
    os.remove(path)

# 7 | Dubbing
---


Dubbing (Synchronisation) bezeichnet den Prozess, bei dem der Originalton eines Films, Videos oder Audioclips durch eine neu eingesprochene Tonspur in einer anderen Sprache ersetzt wird. Dabei wird der übersetzte Text so angepasst, dass er möglichst gut zur Lippenbewegung und Stimmung der Originalaufnahme passt. Ziel ist es, dem Publikum ein möglichst natürliches und immersives Seherlebnis in der eigenen Sprache zu bieten. Dubbing wird häufig bei internationalen Filmproduktionen, Serien oder Videospielen eingesetzt und erfordert eine enge Zusammenarbeit von Übersetzer:innen, Sprecher:innen, Toningenieur:innen und gegebenenfalls KI-gestützten Tools.

| Modell / Anbieter      | API-Zugang möglich? | Hinweise |
| :--------------------- | :------------------ | :------- |
| **ElevenLabs Dubbing** | Ja | Dubbing in 29 Sprachen, erhält Emotion, Timing und Originalstimme. Plattformintegration (z.B. YouTube), <br>automatische Sprechererkennung, professionelle Human Review möglich. |
| **Rask AI**            | Ja | Über 130 Sprachen, Voice Cloning für 29 Sprachen, Multi-Speaker, Lip-Sync, Video-Editing, Integration mit Vimeo/Instagram. |
| **Heygen**             | Ja | 175+ Sprachen/Dialekte, Multi-Speaker, Custom Pronunciation, Brand Consistency, natürliche Intonation, Translate API verfügbar. |
| **D-ID Video Translate** | Ja | Voice Cloning, Lip-Sync, Bulk-Rendering (bis 29 Sprachen), Drag-and-Drop-Interface, API ab $18/Monat. |
| **Murf Dub**           | Ja | 25+ Sprachen, präzise Übersetzung, Voice Cloning, Zeit- und Lip-Sync, Hintergrundmusik bleibt erhalten, API verfügbar, SOC 2/GDPR/ISO-konform. |
| **Synthesys AI**       | Ja | 29 Sprachen, Voice Cloning, Echtzeit-Dubbing, AI-Lip-Sync, 300+ Stimmen, Video-Editing, virtuelle Schauspieler. |
| **Deepdub**            | Ja | 130+ Sprachen, 26+ Emotionen pro Sprecher (eTTS™), Voice Cloning, API für Enterprise, menschliche Qualitätskontrolle, Preis auf Anfrage. |
| **Speechify Studio**   | Ja | 60+ Sprachen, 1.000+ Stimmen, Voice Cloning, Multi-Speaker, Auto-Subtitling, Skript- und Emotionsbearbeitung, Lip-Sync (Beta). |
| **Synthesia**          | Ja | 29 Sprachen, Multi-Speaker, Voice Cloning, Lip-Sync (in Entwicklung), AI-Avatare, einfache Bedienung, API für Unternehmen. |
| **InVideo**            | Ja | 50+ Sprachen, Team-Collaboration, große Voice-Bibliothek, Voice Cloning, Echtzeit-Editing, API verfügbar. |
| **CAMB.AI**            | Ja | 140+ Sprachen, Echtzeit- und Postproduktion, Dubstream für Livestreams, Twitter-Integration, API. |
| **Maestra AI**         | Ja | 125+ Sprachen, Voice Cloning, Multi-Speaker, API, automatische Untertitel, Fokus auf professionelle Inhalte. |
| **Dubverse**           | Ja | Online-Video-Dubbing, TTS, Auto-Subtitles, API, Voice Cloning, Branding, Neo.One & Candy.Two Modelle, einfach integrierbar. |
| **VisualDub**          | Nein (Web-App)      | Studioqualität-LipSync per Web-App, keine öffentliche API dokumentiert. |
| **Dubbing-AI**         | Ja | Automatisiertes Dubbing für Video/Audio, Multi-Language Voiceover, API verfügbar. |


[ElevenLabs Dubbing](https://elevenlabs.io/app/dubbing)

![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/under_construction_dall_e_klein.png)

# 8 | Häufig verwendete Parameter
---

<p><font color='black' size="5">
Zentrale Parameter
</font></p>


<p><font color='blue' size="4">
Allgemeine Parameter
</font></p>

1. **num_inference_steps**:
    
    - Bestimmt die Anzahl der Schritte im Diffusionsprozess
    - Mehr Schritte = bessere Qualität, aber längere Berechnungszeit
    - Typische Werte: 20-50
2. **guidance_scale**:
    
    - Steuert, wie stark der Text-Prompt die Generierung beeinflusst
    - Höhere Werte = stärkere Übereinstimmung mit dem Prompt
    - Typische Werte: 7-9
3. **num_frames**:
    
    - Anzahl der zu generierenden Frames
    - Typische Werte: 16-60 (je nach Modell)
4. **fps** (Frames per Second):
    
    - Bestimmt die Abspielgeschwindigkeit des generierten Videos
    - Typische Werte: 7-30

<p><font color='blue' size="4">
Text-to-Video spezifische Parameter
</font></p>


5. **height/width**:
    - Auflösung des generierten Videos
    - Die optimalen Werte hängen vom verwendeten Modell ab

<p><font color='blue' size="4">
Image-to-Video spezifische Parameter
</font></p>


6. **motion_bucket_id**:
    
    - Steuert die Stärke der Bewegung im Video
    - Bereich: 0-999 (0=minimal, 999=maximal)
7. **noise_aug_strength**:
    
    - Bestimmt, wie stark vom Originalbild abgewichen wird
    - Bereich: 0-1 (0=nah am Original, 1=starke Abweichung)


<p><font color='black' size="5">
Tipps für Einsteiger
</font></p>

1. **Hardware-Anforderungen beachten:**
    
    - Video-Generierung benötigt viel Grafikspeicher (8GB+ VRAM empfohlen)
    - Google Colab kann eine gute Alternative sein
2. **Prompt-Engineering:**
    
    - Detaillierte und klare Prompts führen zu besseren Ergebnissen
    - Experimentiere mit verschiedenen Beschreibungen
3. **Parameter schrittweise anpassen:**
    
    - Beginne mit Standardwerten und passe sie dann an
    - Dokumentiere deine Experimente
4. **Für Image-to-Video:**
    
    - Verwende hochwertige Ausgangsbilder
    - Die besten Ergebnisse erzielst du mit Bildern im Format 768x768 oder 1024x576

# A | Aufgabe
---

Bereit für eine neue Herausforderung? In diesen Aufgaben werden Sie die leistungsstarken Vision-Modelle von **MediaPipe** kennenlernen. Mit diesen Modellen können sie u.a. Objekterkennung, Handgesten und Gesichtsausdrücke in Echtzeit analysieren. Entdecken Sie die Potenziale der GenAI. Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

[MediaPipe](https://mediapipe-studio.webapps.google.com/home)

[MediaPipe Lösungsleitfaden](https://ai.google.dev/edge/mediapipe/solutions/guide?hl=de)

<p><font color='black' size="5">
Video-Objekterkennung
</font></p>

* **Aufgabe:**
    * Installieren Sie die erforderlichen Bibliotheken (MediaPipe, OpenCV).
    * Laden Sie ein vortrainiertes Objekterkennungsmodell von MediaPipe herunter.
    * Schreiben Sie ein Python-Skript, das ein Bild oder einen Videostream einliest und die erkannten Objekte mit Begrenzungsrahmen und Labels visualisiert.
    * Experimentieren Sie mit verschiedenen Bildern und Videostreams und beobachten Sie die Ergebnisse.
* **Zusatzaufgabe:** Versuchen Sie die Genauigkeit der Objekterkennung zu verbessern, indem sie verschiedene Modelle oder Parameter ausprobieren.


<p><font color='black' size="5">
Handgestenerkennung
</font></p>

* **Aufgabe:**
    * Nutzen Sie das MediaPipe-Modell für die Handgestenerkennung.
    * Erstellen Sie ein Programm, das die Positionen der Handgelenke und Finger erkennt und visualisiert.
    * Implementieren Sie eine einfache Gestenerkennung, z. B. die Erkennung von "Daumen hoch" oder "Daumen runter".
    * Lassen Sie Ihr Programm auf erkannte Gesten mit einer Ausgabe im Terminal reagieren.
* **Zusatzaufgabe:** Die Teilnehmer können komplexere Gesten implementieren oder das Programm mit einer anderen Anwendung (z. B. einem Präsentationsprogramm) verbinden, um Gesten zur Steuerung zu verwenden.



<p><font color='black' size="5">
Gesichtserkennung und -verfolgung
</font></p>

* **Aufgabe:**
    * Verwenden Sie das MediaPipe-Modell für die Gesichtserkennung und -verfolgung.
    * Erstellen Sie ein Programm, das Gesichter in einem Videostream erkennt und die Gesichtspunkte (z. B. Augen, Nase, Mund) verfolgt.
    * Implementieren Sie eine einfache Gesichtsausdrucksanalyse, z. B. die Erkennung von Lächeln oder Stirnrunzeln.
    * Die Daten die erhoben werden können visuell in Echtzeit dargestellt werden.
* **Zusatzaufgabe:** Sie können versuchen, die Gesichtsausdrucksanalyse zu verbessern oder das Programm mit einer anderen Anwendung (z. B. einem Avatar-Programm) zu verbinden, um Gesichtsausdrücke auf den Avatar zu übertragen.
